# MAGnet 
### Train your own models to generate audio in python or convert to use in the browser on mimicproject.com

In [5]:
import sys
import tensorflow as tf
import pywt
from utils.audio_dataset_generator import AudioDatasetGenerator
import numpy as np
import tensorflowjs as tfjs

### Set up variables
Including the path to your audio

In [6]:
# Model
load_model           = False

# Dataset
sequence_length      = 40
audio_data_path      = "assets/grime/"
force_new_dataset    = True

# Feature Extraction and Audio Genreation
sample_rate          = 44100
fft_settings         = [2048, 1024, 512]
fft_size             = fft_settings[0]
window_size          = fft_settings[1]
hop_size             = fft_settings[2]

# General Network
learning_rate        = 0.001
amount_epochs        = 100
batch_size           = 64
loss_type            = "mse"
weight_decay         = 0.0001

# Recurrent Neural Network
rnn_type             = "lstm"
number_rnn_layers    = 2
rnn_number_units     = 256

### Make the dataset from the audio

In [ ]:
# Make your dataset

dataset = AudioDatasetGenerator(fft_size, window_size, hop_size,
                                sequence_length, sample_rate)

dataset.load(audio_data_path, force_new_dataset)

In [ ]:
# Set up the model

model = tf.keras.Sequential()

model.add(tf.keras.layers.BatchNormalization(input_shape=[dataset.x_frames.shape[1], dataset.x_frames.shape[2]]))

for layer in range(number_rnn_layers):
    return_sequence = False if layer == (number_rnn_layers - 1) else True
    model.add(tf.keras.layers.LSTM(rnn_number_units, return_sequences= return_sequence))
    
model.add(tf.keras.layers.Dense(dataset.y_frames.shape[1]))

model.add(tf.keras.layers.Activation('linear'))
opt = tf.keras.optimizers.Adam(learning_rate)
model.compile(optimizer=opt, loss=loss_type)

### Train your model

In [ ]:
# Train

model.fit(dataset.x_frames, dataset.y_frames, batch_size=batch_size, epochs=amount_epochs)

### Save your model as a keras model

In [ ]:
# Save your model

model.save("<YOUR MODEL>.h5")

### Convert to use online with tensorflow.js. 
Find example code at https://mimicproject.com/code/b530ba9e-dfd9-0440-8358-86b6420b210d
Upload the 
* .json file
* the shards
* your audio
Update the MODEL_URLS and SAMPLE_URLS in dataset-paths.js

In [ ]:
tfjs.converters.save_keras_model(model, "<YOUR MODEL>.json")

### Or generate samples in python

In [84]:
# Or generate samples in python

amount_samples      = 1
sequence_length_max = 500
impulse_scale       = 1.0
griffin_iterations  = 60
random_chance       = 0.05
random_strength     = 0.0

dimension1 = dataset.x_frames.shape[1]
dimension2 = dataset.x_frames.shape[2]
shape = (1, dimension1, dimension2, 1) if use_cnn else (1, dimension1, dimension2)

audio = []

if use_wavelets:
    temp_audio = np.array(0)
for i in range(amount_samples):                                                                                                                                   
    
    random_index = np.random.randint(0, (len(dataset.x_frames) - 1))                                                                                                                    
                                                                                                                                                                              
    impulse = np.array(dataset.x_frames[random_index]) * impulse_scale
    predicted_magnitudes = impulse
    
    if use_wavelets:
        for seq in range (impulse.shape[0]):
            coeffs = pywt.array_to_coeffs(impulse[seq], dataset.coeff_slices)
            recon = (pywt.waverecn(coeffs, wavelet=wavelet))
            temp_audio = np.append(temp_audio, recon)
    for j in range(sequence_length_max):
        prediction = model.predict(impulse.reshape(shape))
        #Wavelet audio
        if use_wavelets:
            coeffs = pywt.array_to_coeffs(prediction[0], dataset.coeff_slices)
            recon = (pywt.waverecn(coeffs, wavelet=wavelet))
            temp_audio = np.append(temp_audio, recon)
        
        if use_cnn:
            prediction = prediction.reshape(1, dataset.y_frames.shape[1], 1)
        
        predicted_magnitudes = np.vstack((predicted_magnitudes, prediction))    
        impulse = predicted_magnitudes[-sequence_length:]
        
        if (np.random.random_sample() < random_chance) :
            idx = np.random.randint(0, dataset.sequence_length)
            impulse[idx] = impulse[idx] + np.random.random_sample(impulse[idx].shape) * random_strength
        
        done = int(float(i * sequence_length_max + j) / float(amount_samples * sequence_length_max) * 100.0) + 1
        sys.stdout.write('{}% audio generation complete.   \r'.format(done))
        sys.stdout.flush()
        
    if use_wavelets:                                                                                                                                                                        
        audio += [temp_audio]
    else:
        predicted_magnitudes = np.array(predicted_magnitudes).reshape(-1, int(window_size)+1)                                                                           
        audio += [dataset.griffin_lim(predicted_magnitudes.T, griffin_iterations)]
audio = np.array(audio)

In [72]:
# Play them back

from IPython.display import Audio
i = 0
Audio(audio[i], rate=sample_rate)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.
